In [2]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import loadtxt
import os, glob
import geopandas as gpd
import geojsonio
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import math
from IPython.display import Image
from pandas_profiling import ProfileReport

# Plotly
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import cufflinks as cf
from datenguidepy import Query

%matplotlib inline

In [3]:
# Query
q = Query.region('02')
q.add_field('BEV001')
result_df = q.results()

In [4]:
result_df.head()

,id,name,year,BEV001,BEV001_source_title_de,BEV001_source_valid_from,BEV001_source_periodicity,BEV001_source_name,BEV001_source_url
0,02,Hamburg,1995,15872,Statistik der Geburten,1995-01-01T00:00:00,JAEHRLICH,12612,None
1,02,Hamburg,1996,16594,Statistik der Geburten,1995-01-01T00:00:00,JAEHRLICH,12612,None
2,02,Hamburg,1997,16970,Statistik der Geburten,1995-01-01T00:00:00,JAEHRLICH,12612,None
3,02,Hamburg,1998,16235,Statistik der Geburten,1995-01-01T00:00:00,JAEHRLICH,12612,None
4,02,Hamburg,1999,16034,Statistik der Geburten,1995-01-01T00:00:00,JAEHRLICH,12612,None
